<a href="https://colab.research.google.com/github/kakarot98/contextual-chatbot/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tflearn

     |████████████████████████████████| 102kB 2.3MB/s 
  Created wheel for tflearn: filename=tflearn-0.3.2-cp36-none-any.whl size=128208 sha256=1d88edb550d88256363ee6c3fc7aadd54cbc2781647f0cf3e5002b6f9bce6aca
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [2]:
!pip install tensorflow==1.14


     |████████████████████████████████| 109.2MB 49kB/s 
     |████████████████████████████████| 491kB 43.2MB/s 
     |████████████████████████████████| 3.2MB 45.4MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [3]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye"],\r\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n        },\r\n        {"tag": "hours",\r\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n        },\r\n        {"tag": "location",\r\n         "patterns": 

In [5]:
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [ ]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [6]:
words = []
classes = []
ignore = ['?'] #to ignore punctuations
documents = []
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #tokenizing words in sentences
    w = nltk.word_tokenize(pattern)
    #addidng to words list
    words.extend(w)
    #add words to doc
    documents.append((w, intent['tag']))
    #add tags to classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [7]:
#stemming brings down a word to its base form and set takes out duplicate words
words = [stemmer.stem(w.lower()) for w in words if w not in ignore] #the for loop would check if word is in words list and not in ignore list
words = sorted(list(set(classes)))

print(len(documents), "documents", documents)
print(len(classes), "classes:", classes)
print(len(words), "stemmed words:", words)

31 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['What', 'is', 'your', 'location', '?'], 'location'), (['Where', 'are', 'you', 'located', '?'], 'location'), (['What', 'is', 'your', 'address', '?'], 'location'), (['Where', 'is', 'your', 'restaurant', 'situated', '?'], 'location'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['What', 'is', 'your', 'menu', 'for', 'today', '?'], 'todaysmenu'), (['What', 'are', '

In [8]:
training = []
output = []
#here we will create training data
output_empty = [0] * len(classes)

#creating bag of words for each sentence
for doc in documents:
  bag=[]  #initilise bag of  words
  pattern_words = doc[0]  #list of tokenized words for pattern
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  #stemming each word
  #create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  #output is 1 for cuurent tag and 0 for rest of tags
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

#shuffling features
random.shuffle(training)
training = np.array(training)

#divide data
train_x = list(training[:,0])
train_y = list(training[:,1])



In [9]:
#reset underlyin graph
tf.reset_default_graph()

#adding layers
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

#define model and set up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

#training of model
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 2.01478 | time: 0.012s
| Adam | epoch: 1000 | loss: 2.01478 - acc: 0.2183 -- iter: 24/31
Training Step: 4000  | total loss: 1.98756 | time: 0.017s
| Adam | epoch: 1000 | loss: 1.98756 - acc: 0.2394 -- iter: 31/31
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [10]:
import pickle
pickle.dump({'words':words, 'classes':classes, 'train_x': train_x, 'train_y':train_y}, open("training_data", "wb"))

In [11]:
#restoring all the data
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [12]:
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [13]:
#loading the model again
model.load('./model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [15]:
def clean_up_sentence(sentence):
  #toeknize the words first
  sentence_words = nltk.word_tokenize(sentence)
  #stemming each word
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

#return bag of words array
def bow(sentence, words, show_details=False):
  #tokeinze pattern
  sentence_words = clean_up_sentence(sentence)
  #generate bag of words(bow)
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        bag[i] = s
        if show_details:
          print("found in bag: %s %w")
  return(np.array(bag))

In [16]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
  #generate probability
  results = model.predict([bow(sentence, words)])[0]
  #filter out probabilty below threshold
  results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
  #sort by strength
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  #return tuple of intent and prob
  return return_list

def response(sentence, userID='123', show_details=False):
  results = classify(sentence)
  #if it has a classification then find matching intent
  if results:
    #loop as long as there are matches to process
    while results:
      for i in intents['intents']:
        #find a tag matching the first result
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))
      results.pop(0)

In [17]:
classify('What are your hours of operation?')

[]